#### Set up with some Dummy Emails
In real life these would be ingested from a service or a storage location

In [3]:
email_messages = [
    """
    Hey, want me to grab you a drink on my way in this morning?
    """,
    """
    I'm going to be late this morning, can you take the 10 o'clock?
    """,
    """
    I'm going to the coffee shop, want anything?
    """,
    """
    Can you cover the meeting tomorrow? I have an appointment.
    """
    ]

negative = 2
positive = 1

Python SDK for Lex Runtime at https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lex-runtime.html

Note that the SDKs are all available in multiple programming languages https://aws.amazon.com/tools/

In [4]:
import json
import datetime
import boto3
lex = boto3.client('lex-runtime')

#### Call Lex

In [5]:
email_intent = lex.post_text(
    botName='auto_message_bot',
    botAlias='test',
    userId='123',
    inputText=email_messages[0]
)
print(json.dumps(email_intent, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "03bc7677-2b5b-4dad-89ff-5ac04ba0fd0d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "03bc7677-2b5b-4dad-89ff-5ac04ba0fd0d",
      "date": "Thu, 23 Jul 2020 23:51:01 GMT",
      "content-type": "application/json",
      "content-length": "400"
    },
    "RetryAttempts": 0
  },
  "intentName": "GetCoffee",
  "slots": {},
  "sentimentResponse": {
    "sentimentLabel": "NEUTRAL",
    "sentimentScore": "{Positive: 0.13963953,Negative: 0.004401778,Neutral: 0.85589135,Mixed: 6.733458E-5}"
  },
  "dialogState": "ReadyForFulfillment",
  "sessionId": "2020-07-23T23:50:58.691Z-IAhkyPAm"
}


#### Loop through emails to determine Intent for each and take appropriate action

In [6]:
for email in email_messages:
    
    print(email)
    
    email_intent = lex.post_text(
        botName='auto_message_bot',
        botAlias='test',
        userId='123',
        inputText=email
    )
    
    print(email_intent['intentName'])
    
    if email_intent['intentName'] == 'GetCoffee':
        # Set up an auto reply - we don't want to miss coffee!
        print('Auto Reply: YESSS!!')
    
    elif email_intent['intentName'] == 'CoverForMe':
        # Add to Insights gained from Comprehend
        print('Flag as Important')


    print('')
    print('-------')


    Hey, want me to grab you a drink on my way in this morning?
    
GetCoffee
Auto Reply: YESSS!!

-------

    I'm going to be late this morning, can you take the 10 o'clock?
    
CoverForMe
Flag as Important

-------

    I'm going to the coffee shop, want anything?
    
GetCoffee
Auto Reply: YESSS!!

-------

    Can you cover the meeting tomorrow? I have an appointment.
    
CoverForMe
Flag as Important

-------


#### Getting a little fancy - Customize Coffee Order based on overall sentiment of emails in mailbox 

In [7]:
for email in email_messages:
    
    print(email)
    
    email_intent = lex.post_text(
        botName='auto_message_bot',
        botAlias='test',
        userId='123',
        inputText=email
    )
    
    print(email_intent['intentName'])
    
    if email_intent['intentName'] == 'GetCoffee':
        # Set up an auto reply - we don't want to miss coffee!
        # Get sentiment from insights from comprehend (mocked here)
        if positive > negative:
            # Chill morning
            print('Reply: Yes, I would love a Green Tea!')
        else:
            # Need that extra boost!
            print('Reply: YESSS!! An Americano with extra extra shots please!')
    
    elif email_intent['intentName'] == 'CoverForMe':
        # Add to Insights gained from Comprehend
        print('Flag as Important')


    print('')
    print('-------')


    Hey, want me to grab you a drink on my way in this morning?
    
GetCoffee
Reply: YESSS!! An Americano with extra extra shots please!

-------

    I'm going to be late this morning, can you take the 10 o'clock?
    
CoverForMe
Flag as Important

-------

    I'm going to the coffee shop, want anything?
    
GetCoffee
Reply: YESSS!! An Americano with extra extra shots please!

-------

    Can you cover the meeting tomorrow? I have an appointment.
    
CoverForMe
Flag as Important

-------


#### Getting even fancier with Dates

In [8]:
for email in email_messages:
    
    print(email)
    
    email_intent = lex.post_text(
        botName='auto_message_bot',
        botAlias='test',
        userId='123',
        inputText=email
    )
    
    print(email_intent['intentName'])
    
    if email_intent['intentName'] == 'GetCoffee':
        # Set up an auto reply - we don't want to miss coffee!
        # Get overall sentiment for all emails we have saved from earlier (mocked here)
        if positive > negative:
            # Chill morning
            print('Reply: Yes, I would love a Green Tea!')
        else:
            # Need that extra boost!
            print('Reply: YESSS!! An Americano with extra extra shots please!')
    
    elif email_intent['intentName'] == 'CoverForMe':
        # We can check the date captured in the Lex Slot
        cover_date_string = email_intent['slots']['date']
        
        if cover_date_string != None:
            # Check if the date is today
            cover_date = datetime.datetime.strptime(cover_date_string, "%Y-%m-%d")
            today = datetime.datetime.now()
            
            if cover_date.date() == today.date():
                # Add to Insights gained from Comprehend - make sure we see it right away
                print('Flag as Important')
                
            else:
                print('//it can wait')
        else:
            print('//it can wait')


    print('')
    print('-------')


    Hey, want me to grab you a drink on my way in this morning?
    
GetCoffee
Reply: YESSS!! An Americano with extra extra shots please!

-------

    I'm going to be late this morning, can you take the 10 o'clock?
    
CoverForMe
Flag as Important

-------

    I'm going to the coffee shop, want anything?
    
GetCoffee
Reply: YESSS!! An Americano with extra extra shots please!

-------

    Can you cover the meeting tomorrow? I have an appointment.
    
CoverForMe
//it can wait

-------


|￣￣￣￣￣￣|
| PLS HELP,|
|  NEED    |
|  COFFEE! |
|＿＿＿＿＿＿|
(\__/) ||
(•ㅅ•) ||
/ 　 づ